In [33]:
# DSC670 - Week7 - Exercise  - Tuning and Adapting LLMs Part II

Techniques such as LoRA take considerable time and computational resources (including the need for a GPU). For this week’s assignment, re-build the model from the Matthew MacFarquhar article.

https://blog.devgenius.io/sculpting-language-gpt-2-fine-tuning-with-lora-1caf3bfbc3c6


Since a GPU is required, please consider using Google Colab. You can change the runtime type to GPU (T4). It doesn’t take long to train the model.



Build the solution using a Jupyter Notebook (no other Python is acceptable) and include Markdown that explains what the author is doing in the code. I’m looking for your commentary and evaluation in your own words.content)cise Code.txt

## ------------------------------------------
## Import necessary libraries
## ------------------------------------------
##  import torch

Imports PyTorch, a deep learning framework used for:

Building and training neural networks

Handling tensors (multi-dimensional arrays)

Running computations on GPUs or CPUs

In this script, PyTorch is used for:

Moving models and data to GPU (model.to(device))

Saving model weights (torch.save(...))

Disabling gradient computation (with torch.no_grad():)

# from transformers import (...)

This imports several key tools from Hugging Face Transformers, a popular library for working with pre-trained language models like GPT-2, BERT, etc.

## AutoTokenizer

Automatically downloads and loads the correct tokenizer for the model name (e.g., "gpt2").

Tokenizers convert raw text into token IDs (numbers) that the model understands, and back into text.

## AutoModelForCausalLM

Automatically loads a pre-trained language model suitable for causal language modeling (i.e., predicting the next word).

Example: GPT-2, GPT-Neo, etc.

"CausalLM" stands for Causal Language Model, which is trained to generate text.

## Trainer

A high-level training utility from Hugging Face that simplifies model training.

It handles gradient computation, batching, optimization, evaluation, and logging automatically.

## TrainingArguments

A configuration class that defines training hyperparameters, such as:

batch_size, learning_rate, max_steps, etc.

output_dir (where model checkpoints are saved)

logging_steps (how often to print training progress)

## DataCollatorForLanguageModeling

A utility that prepares batches of tokenized text for language modeling.

It ensures that input sequences are padded to equal length in each batch and optionally masks tokens for masked LM (not used here since GPT-2 is causal).

## from peft import LoraConfig, get_peft_model

Imports tools from the PEFT (Parameter-Efficient Fine-Tuning) library.

PEFT allows you to fine-tune very large models (like GPT-2) without updating all parameters — only small adapter layers (LoRA layers) are trained.

## LoraConfig

Configuration object that defines LoRA settings such as:

r (rank of low-rank matrices)

lora_alpha (scaling factor)

lora_dropout (dropout rate)

task_type (e.g., "CAUSAL_LM")

## get_peft_model

Takes a base model and a LoraConfig, then wraps the model with LoRA layers.

The resulting model trains only a small number of new parameters — much faster and more memory-efficient.

## from datasets import load_dataset

Imports the Hugging Face Datasets library function used to load and preprocess datasets easily.

load_dataset() automatically downloads datasets from the Hugging Face Hub and returns them in a PyTorch-friendly format.

Example:

data = load_dataset("Abirate/english_quotes")


This loads a dataset of English quotes for training.


In [34]:
# ==========================================
# LoRA Fine-Tuning and Text Generation with GPT-2
# ==========================================
# This Jupyter Notebook combines model training and inference into one file.
# It fine-tunes GPT-2 using Low-Rank Adaptation (LoRA) on a dataset of English quotes,
# then generates text completions using the fine-tuned model.

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

# Imports tools from the PEFT (Parameter-Efficient Fine-Tuning) library.

PEFT allows you to fine-tune very large models (like GPT-2) without updating all parameters — only small adapter layers (LoRA layers) are trained.

#LoraConfig

Configuration object that defines LoRA settings such as:

r (rank of low-rank matrices)

lora_alpha (scaling factor)

lora_dropout (dropout rate)

task_type (e.g., "CAUSAL_LM")

#get_peft_model

Takes a base model and a LoraConfig, then wraps the model with LoRA layers.

The resulting model trains only a small number of new parameters — much faster and more memory-efficient.

#4. from datasets import load_dataset

Imports the Hugging Face Datasets library function used to load and preprocess datasets easily.

load_dataset() automatically downloads datasets from the Hugging Face Hub and returns them in a PyTorch-friendly format.

Example:

data = load_dataset("Abirate/english_quotes")


This loads a dataset of English quotes for training.

In [ ]:
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [35]:
# ------------------------------------------
# Step2: Setup device (GPU if available), check whether your computer or environment has a GPU that supports CUDA (NVIDIA’s GPU computing platform).

# 'cuda' - run computations on the GPU for faster training.

# 'cpu' - fallback if GPU is not available.
# ------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")



Using device: cuda


In [36]:
# ------------------------------------------
# Step: 3 Load pre-trained GPT-2 model and tokenizer
# ------------------------------------------
model_name = "gpt2" # "gpt2" is a model name recognized by Hugging Face’s Transformers library.

 # Automatically choose the correct model architecture for causal (left-to-right) language modeling (used for text generation).
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')

#The tokenizer converts text into tokens (numerical IDs) that the model understands, and vice versa (for decoding output back into text).
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a pad token, so we set it to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token



In [37]:
# ------------------------------------------
# Step: 4 Freeze base model parameters
# ------------------------------------------
# This ensures that only LoRA layers will be trained. fine-tuning or LoRA adaptation setups
# This line freezes the pretrained GPT-2 model, making sure its existing weights stay unchanged while you train only a small number of new parameters (for example, LoRA layers).
for param in model.parameters():
    param.requires_grad = False



In [38]:
# ------------------------------------------
# Step: 5 Configure and attach LoRA adapter
# ------------------------------------------
# LoRA adds trainable rank-decomposition matrices to attention layers,
# reducing the number of trainable parameters significantly.
config = LoraConfig(
    r=16,                 # Rank of LoRA matrices
    lora_alpha=32,        # Scaling factor, Controls how strongly LoRA updates affect the base model
    lora_dropout=0.05,    # Dropout for regularization, Randomly drops 5% of LoRA connections during training to reduce overfitting
    bias="none",          # Keeps GPT-2 biases frozen
    task_type="CAUSAL_LM" # Language modeling task, Ensures LoRA modifies the right layers for autoregressive text generation
)

model = get_peft_model(model, config) # Makes only those LoRA parameters trainable, while the rest of GPT-2 remains frozen.



/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [28]:
# ------------------------------------------
# Step: 6 Load and preprocess dataset
# ------------------------------------------
# Using a small public dataset of English quotes from Hugging Face Datasets
# Prepares the training dataset so it can be used to fine-tune the GPT-2 model (with LoRA).
#creates a single text field that the model can learn from (i.e., input → output structure).
# Later, we can have the model predict tags from quotes, or continue learning language style.
data = load_dataset("Abirate/english_quotes") # Downloads a public dataset from Hugging Face Hub named "Abirate/english_quotes"

# Combine 'quote' and 'tags' columns into a single training text
def merge_columns(entry):
    entry["prediction"] = entry["quote"] + " ->: " + str(entry["tags"]) # The new combined text is stored in a new column called "prediction"
    return entry

data['train'] = data['train'].map(merge_columns)

# Tokenize the text for model input, This prepares the dataset for training.
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)



In [29]:
# ------------------------------------------
# Step 7: Helper function: Display trainable parameter count
# ------------------------------------------
def print_trainable_parameters(model):
    """
    Prints the number of trainable vs total parameters in the model.
    Helps confirm LoRA reduces training load.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} | All params: {all_param} | "
        f"Trainable%: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(model)



Trainable params: 589824 | All params: 125029632 | Trainable%: 0.47%


In [30]:
# ------------------------------------------
# Step 8: Training setup
# this block sets up the training configuration and initializes the trainer that will fine-tune your LoRA-adapted GPT-2 model#
# ------------------------------------------
# Using the Hugging Face Trainer API for simplicity
trainer = Trainer(
    model=model,
    train_dataset=data['train'],
    args=TrainingArguments(
        per_device_train_batch_size=4, # Number of examples processed on each GPU (or CPU) before gradient accumulation. Smaller batches reduce memory use.
        gradient_accumulation_steps=4, # Accumulates gradients over 4 steps before updating weights — effectively making the effective batch size = 4 × 4 = 16.
        warmup_steps=100, # Slowly increases the learning rate during the first 100 steps to stabilize training.
        max_steps=500, # Stops training after 500 optimization steps (instead of using epochs).
        learning_rate=2e-4, # Sets the learning rate for the AdamW optimizer.
        logging_steps=50, # Logs training loss every 50 steps.
        output_dir='outputs', # Directory where checkpoints, logs, and model outputs will be saved.
        auto_find_batch_size=True, # Automatically adjusts the batch size if GPU memory is insufficient.
        report_to="none",          # disables W&B and other trackers, Prevents integration with external logging tools (like Weights & Biases or TensorBoard).
        logging_dir=None,          # Avoids creating empty log directories when logging is disabled.
    ),
    # the model learns to predict the next word in a sequence (like GPT-2 normally does), not to fill in masked words (which would be Masked LM like BERT).
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # A data collator organizes the dataset samples into batches during training.
)

# Disable caching during training, caching can cause shape mismatches or extra memory usage — so it’s turned off for safety.
model.config.use_cache = False



The model is already on multiple devices. Skipping the move to device specified in `args`.


In [31]:
# ------------------------------------------
# Step 9: Train the LoRA-augmented model
# actual training and saving step for your LoRA-adapted GPT-2 model


#Calls the Hugging Face Trainer to train the model using your dataset, tokenizer, and training arguments.
#What happens under the hood:
#Batches are created from data['train'] using the DataCollator.
#LoRA parameters in the model are updated via backpropagation.
#Gradients accumulate according to gradient_accumulation_steps.
#Optimizer updates the trainable parameters (LoRA layers only).
#Logging occurs every logging_steps (here, every 50 steps).
#Training stops automatically after max_steps=500 or when interrupted.#
# ------------------------------------------


trainer.train()

# Save LoRA fine-tuned weights only (not full GPT-2 model), The base GPT-2 parameters are frozen, so only the small LoRA layers are being trained.
# This makes training much faster and lighter on GPU memory.
torch.save(model.state_dict(), 'lora.pt')
print("LoRA fine-tuned weights saved as 'lora.pt'")



Step,Training Loss
50,4.204800
100,3.429200
150,2.983800
200,2.888900
250,2.846000
300,2.853000
350,2.808700
400,2.841400
450,2.816200
500,2.820200


✅ LoRA fine-tuned weights saved as 'lora.pt'


In [32]:
# ------------------------------------------
# Step 10: Inference: Generate text using fine-tuned model
# Use the LoRA-adapted GPT-2 for inference (text generation) after training
# ------------------------------------------
# Re-load LoRA weights to confirm inference works separately from training, Loads the previously saved LoRA weights from disk.
model.load_state_dict(torch.load("lora.pt", map_location=device)),
model = model.to(device)

# Example input prompt for generation
prompt = "“Life is like a box of chocolates, you never know what you are gonna get” ->: "

# Tokenize and move to device, Converts the text prompt into token IDs and an attention mask.
#return_tensors='pt' returns PyTorch tensors.
#.to(device) moves the tensors to the GPU (or CPU) to match the model’s device.
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# Generate prediction, generates up to 25 new tokens., torch.no_grad disables gradient calculation — saves memory and speeds up inference.
# Only the LoRA layers affect generation since the base model is frozen.
with torch.no_grad():
    output_tokens = model.generate(**inputs, max_new_tokens=25)

# Decode and print output text, converts token IDs back into human-readable text.
print("\n Model Output:\n")
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True)) #skip_special_tokens=True removes special tokens like `` from the output.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📝 Model Output:

“Life is like a box of chocolates, you never know what you are gonna get” ->: vernacular,chocolates,chocolates-and-chocolates-in-the-world,chocol


## Conclusion: 
In this project, I have successfully applied Low-Rank Adaptation (LoRA) to fine-tune a pre-trained GPT-2 model for generating text based on English quotes and their associated tags. By freezing the original GPT-2 parameters and training only the small LoRA adapter layers, we achieved parameter-efficient fine-tuning, which significantly reduces computational resources and memory requirements.

The workflow demonstrated how to:

Prepare and tokenize a dataset for language modeling.

Configure LoRA parameters and integrate them into a large language model.

Train the model using Hugging Face’s Trainer with gradient accumulation and proper batching.

Save only the fine-tuned LoRA weights, allowing for efficient storage and future reuse.

Generate new text using the fine-tuned model, illustrating how the model learned to produce meaningful continuations based on the prompt.

Overall, this approach shows that large language models can be effectively adapted to specific tasks without retraining billions of parameters, making it feasible to fine-tune on smaller datasets and limited hardware. LoRA provides a practical, scalable, and resource-efficient solution for customizing pretrained models for specialized applications.

While models like ChatGPT-3 and ChatGPT-4 could likely perform quote tagging with few- or zero-shot learning, they are vastly larger than GPT-2 and require massive computational resources. For context, GPT-4 has around 1 trillion parameters, GPT-3 has 175 billion, and GPT-2 has 1.5 billion—already challenging for CPU inference.

For lightweight, task-specific applications, fine-tuning a smaller model like GPT-2 is often more practical and efficient, allowing you to run the model locally without relying on overpowered, resource-intensive models like ChatGPT.